In [10]:
# Cell 1: Imports and Logging Setup
import os
import pandas as pd
import polars as pl
import torch
import numpy as np
import logging
import json
import joblib
from kaggle_evaluation import mitsui_inference_server

# Import custom modules (adjust paths if needed)
from models import load_ckp, FEDForecaster, load_timesfm_model
from adapters import FEDAdapter, TimesFMAdapter
from utils import weighted_ensemble
from dataprep import impute_features_with_staleness  # Added for preprocessing

# Constants
NUM_TARGET_COLUMNS = 424
CHECKPOINT_DIR = "../checkpoints/"

# Global state for persistence
loaded = False
adapters = []  # List of (adapter, model_type, checkpoint_path) tuples
history = pl.DataFrame()  # Persistent price/feature history
feature_cols_list = []  # Per-model feature columns
target_cols = None  # Shared target columns
input_len = 512  # Default; updated per model

# Logging setup
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.FileHandler('inference_log.log'), logging.StreamHandler()]
)

logging.info("Environment setup complete.")

ModuleNotFoundError: No module named 'grpc'

In [8]:
def predict(
    test: pl.DataFrame,
    label_lags_1_batch: pl.DataFrame,
    label_lags_2_batch: pl.DataFrame,
    label_lags_3_batch: pl.DataFrame,
    label_lags_4_batch: pl.DataFrame,
) -> pl.DataFrame:
    """
    Ensemble predictions from FED and TimesFM models with preprocessing.
    """
    global loaded, adapters, history, feature_cols_list, target_cols, input_len

    if not loaded:
        try:
            checkpoints = [
                ("FED", "FED_trial7_20251004_185519.pt", FEDForecaster, FEDAdapter),
                ("TimesFM", "timesfm/finetuned_target_0/finetuned.pt", None, TimesFMAdapter)
            ]

            for model_type, ckpt_file, model_class, adapter_class in checkpoints:
                ckpt_path = os.path.join(CHECKPOINT_DIR, ckpt_file)
                if not os.path.exists(ckpt_path):
                    logging.error(f"Checkpoint {ckpt_path} not found")
                    continue

                if model_type == "FED":
                    model, scaler, feat_cols, tgt_cols, model_params = load_ckp(
                        dir=os.path.dirname(ckpt_path),
                        model_class=model_class,
                        model_kwargs=model_params if os.path.exists(os.path.join(os.path.dirname(ckpt_path), "model_params.json")) else {},
                        device='cuda' if torch.cuda.is_available() else 'cpu'
                    )
                    adapter = adapter_class(model, scaler, tgt_cols)
                    current_input_len = model_params.get('input_len', input_len)
                else:  # TimesFM
                    model = load_timesfm_model(
                        model_name="google/timesfm-1.0-200m-pytorch",
                        device='cuda' if torch.cuda.is_available() else 'cpu',
                        horizon_len=1,
                        context_len=512,
                        batch_size=32
                    )
                    state_dict = torch.load(ckpt_path, map_location=model.device)
                    model.load_state_dict(state_dict)
                    model.eval()
                    tgt_cols_path = os.path.join(os.path.dirname(ckpt_path), "target_cols.json")
                    tgt_cols = json.load(open(tgt_cols_path)) if os.path.exists(tgt_cols_path) else [f"target_{i}" for i in range(NUM_TARGET_COLUMNS)]
                    feat_cols = tgt_cols
                    scaler = joblib.load(os.path.join(os.path.dirname(ckpt_path), "scaler.pkl")) if os.path.exists(os.path.join(os.path.dirname(ckpt_path), "scaler.pkl")) else None
                    adapter = adapter_class(model, scaler, tgt_cols, context_length=512, horizon_length=1, freq_type=0)
                    current_input_len = 512

                if target_cols is None:
                    target_cols = tgt_cols
                elif target_cols != tgt_cols:
                    logging.warning(f"Target cols mismatch in {ckpt_file}; expected {target_cols}, got {tgt_cols}")
                    continue

                adapters.append((adapter, model_type, ckpt_path))
                feature_cols_list.append(feat_cols)
                input_len = max(input_len, current_input_len)
                logging.info(f"Loaded {model_type} from {ckpt_path}, features: {len(feat_cols)}, targets: {len(tgt_cols)}")

            if not adapters:
                raise ValueError("No valid models loaded for ensembling")
            loaded = True
        except Exception as e:
            logging.error(f"Model loading failed: {str(e)}")
            return pl.DataFrame({f'target_{i}': [0.0] for i in range(NUM_TARGET_COLUMNS)})

    # Preprocess test batch
    test_pd = test.to_pandas()
    X_imputed, _, _ = impute_features_with_staleness(test_pd, date_col="date_id", cap_days=None)
    test_processed = pl.from_pandas(X_imputed)

    # Append to history
    history = pl.concat([history, test_processed])

    # Ensure sufficient history
    if len(history) < input_len:
        pad_rows = input_len - len(history)
        pad_dict = {col: [0.0] * pad_rows for col in history.columns if col != 'date_id'}
        if 'date_id' in history.columns:
            last_date = history['date_id'][-1] if len(history) > 0 else 0
            pad_dict['date_id'] = list(range(last_date - pad_rows, last_date))
        pad_df = pl.DataFrame(pad_dict)
        history = pl.concat([pad_df, history])

    # Generate predictions
    predictions = []
    date_id = history['date_id'][-1] if 'date_id' in history.columns else None
    for (adapter, model_type, ckpt_path), feat_cols in zip(adapters, feature_cols_list):
        try:
            recent_history = history.tail(input_len)
            X_hist_pd = recent_history.select(feat_cols + ['date_id']).to_pandas()
            pred, metadata = adapter.predict_next(X_hist_pd, Y_hist=None, input_len=input_len, lag=0)
            if not np.isfinite(pred).all():
                logging.warning(f"Non-finite predictions from {model_type}, replacing with zeros")
                pred = np.zeros(NUM_TARGET_COLUMNS, dtype=float)
            predictions.append(pred)
            logging.info(f"{model_type} predicted for date_id {date_id}: {pred[:5]}...")
        except Exception as e:
            logging.error(f"{model_type} prediction failed: {str(e)}")
            predictions.append(np.zeros(NUM_TARGET_COLUMNS, dtype=float))

    # Ensemble predictions
    weights = np.ones(len(predictions)) / len(predictions)
    final_pred = weighted_ensemble(*predictions, w=weights)

    # Convert to pl.DataFrame
    pred_dict = {f'target_{i}': [float(final_pred[i])] for i in range(NUM_TARGET_COLUMNS)}
    return pl.DataFrame(pred_dict)

In [ ]:
# Inference server
inference_server = mitsui_inference_server.MitsuiInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(('/kaggle/input/mitsui-commodity-prediction-challenge/',))

AttributeError: module 'kaggle_evaluation' has no attribute 'mitsui_inference_server'

----
All-in-one

In [ ]:
import os
import pandas as pd
import polars as pl
import torch
import logging
import json
import joblib
import kaggle_evaluation.mitsui_inference_server

# Import your custom modules (adjust paths as needed; assuming they are in /root/src or current dir)
from models import load_ckp, LSTMForecaster  # Replace LSTMForecaster with your model class if different
from adapters import LSTMAdapter  # Replace with your specific adapter (e.g., FEDAdapter, TCNAdapter)
from configs import TrainConfig  # If needed for defaults

NUM_TARGET_COLUMNS = 424

# Global state for persistence across calls (loaded once in first predict call)
loaded = False
adapter = None
history = pl.DataFrame()  # Persistent price/feature history (appended per batch)
feature_cols = None  # Will be loaded from checkpoint
target_cols = None  # Will be loaded from checkpoint
input_len = 64  # Default; will be updated from model_params if available

# Set up basic logging (optional, for debugging; logs to file/stderr)
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.FileHandler('inference_log.log'), logging.StreamHandler()]
)

def predict(
    test: pl.DataFrame,
    label_lags_1_batch: pl.DataFrame,
    label_lags_2_batch: pl.DataFrame,
    label_lags_3_batch: pl.DataFrame,
    label_lags_4_batch: pl.DataFrame,
) -> pl.DataFrame:
    """
    Inference function: Predicts all 424 targets for the current date_id batch.
    - Loads model/adapter once (in first call, no time limit).
    - Appends new test features (prices) to persistent history.
    - Prepares X_hist from history (last input_len rows).
    - Uses adapter to predict next targets (forecast for current date_id).
    - Ignores lagged labels (as current code doesn't use past targets as features).
    - Returns pl.DataFrame with one row: target_0 to target_423.
    """
    global loaded, adapter, history, feature_cols, target_cols, input_len

    if not loaded:
        try:
            # Load checkpoint (adjust path; e.g., select best model file)
            ckpt_dir = "/root/checkpoints/"  # Or os.environ.get('CHECKPOINT_DIR', './checkpoints/')
            model_class = LSTMForecaster  # Replace with your model (e.g., FEDForecaster)
            model_kwargs = {}  # Base kwargs; load_ckp will use model_params.json if present

            model, scaler, feature_cols, target_cols, model_params = load_ckp(
                dir=ckpt_dir,
                model_class=model_class,
                model_kwargs=model_kwargs,
                device='cuda' if torch.cuda.is_available() else 'cpu'
            )

            # Update input_len from model_params (if saved)
            input_len = model_params.get('input_len', input_len)

            # Create adapter (replace LSTMAdapter with your model's adapter)
            adapter = LSTMAdapter(model, scaler, target_cols)

            loaded = True
            logging.info(f"Model loaded from {ckpt_dir}. Feature cols: {len(feature_cols)}, Target cols: {len(target_cols)}")
        except Exception as e:
            logging.error(f"Model loading failed: {str(e)}")
            # Fallback to dummy predictions on error
            return pl.DataFrame({f'target_{i}': [0.0] for i in range(NUM_TARGET_COLUMNS)})

    # Append current test batch to history (assumes test has date_id and feature columns)
    history = pl.concat([history, test])

    # Ensure history has enough rows; pad with zeros if too short (rare, but defensive)
    if len(history) < input_len:
        pad_rows = input_len - len(history)
        pad_dict = {col: [0.0] * pad_rows for col in history.columns if col != 'date_id'}
        if 'date_id' in history.columns:
            last_date = history['date_id'][-1] if len(history) > 0 else 0
            pad_dict['date_id'] = list(range(last_date - pad_rows, last_date))
        pad_df = pl.DataFrame(pad_dict)
        history = pl.concat([pad_df, history])

    # Prepare X_hist as pd.DataFrame for adapter (select last input_len rows, feature_cols + date_id)
    if 'date_id' not in history.columns:
        history = history.with_columns(pl.Series(name='date_id', values=range(len(history))).alias('date_id'))
    recent_history = history.tail(input_len)
    X_hist_pd = recent_history.select(feature_cols + ['date_id']).to_pandas()

    # Predict (lag=0 since predicting for current; adjust if your setup uses lag for horizon)
    try:
        pred, metadata = adapter.predict_next(X_hist_pd, Y_hist=None, input_len=input_len, lag=0)
        logging.info(f"Predicted for date_id {metadata.get('date_id')}: {pred[:5]}...")
    except Exception as e:
        logging.error(f"Prediction failed: {str(e)}")
        pred = np.zeros(NUM_TARGET_COLUMNS, dtype=float)

    # Convert predictions to pl.DataFrame (one row)
    pred_dict = {f'target_{i}': [pred[i]] for i in range(NUM_TARGET_COLUMNS)}
    return pl.DataFrame(pred_dict)



ModuleNotFoundError: No module named 'kaggle_evaluation'

In [ ]:
# Set up the inference server
inference_server = kaggle_evaluation.mitsui_inference_server.MitsuiInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(('/kaggle/input/mitsui-commodity-prediction-challenge/',))